# Case Study A/B Testing 

<img src="./img/head_img.jpg" style="width:50%;height=50%"/>

The present case study is a part of the Nano-degree program of Udacity. 
Within this case study, the goal is to revise the structure of a homepage of a company in order to increase the number of people that download the software of this mentioned company as well as ultimately increase the number of licences purchased. 
 
The company's Website has five main sections: 
1. the homepage; 
2. a section with additional information. gallery, and examples; 
3. a page for users to download the software; 
4. a page for users to purchase a license; and 
5. a support subsite with documentation and FAQs software; 

For the software itself, the Website requires that users create an account in order to download the software program. The program is usable freely for seven days after download.
When the trial period is hit, the program will bring up dialog box that takes the 
user to the license page. After purchasing a license, the user will receive a unique Code associated with their Site account. This Code can then be used with the program to register it with that user, and the program can be used thereafter without issue. 

The goal is to leverage A/B Testing to check if the download rate and the licence purchase could be raised through revising the structure of the homepage.

Concretly we:
* Explore the data
* Execute A/B Testing on the data to check if the homepage structure has a significant influence on the above mentioned mesasures



## Import libraries + load the data

Import the relevant libraries to make a Hypothesis Test as well as load the data

In [23]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats import proportion as proptests
import numpy as np

In [24]:
df = pd.read_csv("homepage-experiment-data.csv")

## Check the data

In [25]:
df.head()

,Day,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
0,1,1764,246,1,1850,339,3
1,2,1541,234,2,1590,281,2
2,3,1457,240,1,1515,274,1
3,4,1587,224,1,1541,284,2
4,5,1606,253,2,1643,292,3


In [26]:
df.describe()

,Day,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
count,29.000000,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000
mean,15.000000,1615.551724,260.482759,24.482759,1632.62069,294.758621,25.241379
std,8.514693,116.308268,28.338037,13.873461,113.02636,22.404807,13.762410
min,1.000000,1457.000000,223.000000,1.000000,1458.00000,256.000000,1.000000
25%,8.000000,1529.000000,240.000000,12.000000,1555.00000,279.000000,20.000000
50%,15.000000,1602.000000,254.000000,30.000000,1606.00000,290.000000,29.000000
75%,22.000000,1700.000000,276.000000,34.000000,1728.00000,300.000000,36.000000
max,29.000000,1822.000000,331.000000,42.000000,1861.00000,349.000000,44.000000


## A/A Testing: Invariant Matric

A prerequisite for A/B Testing is to insure that the groups included in the hypothesis test are in terms of numbers equally high.<br>
In case of significantly differering group size, we can't proceed to A/B Testing because the results could be biased.

So we're applying A/A Testing under the following hypthesis:

$H_0: cust_{ctrl} = cust_{treat} $ <br>
$H_1: cust_{ctrl} \neq cust_{treat} $ <br>
$ p-val$ $assumption $: 0.5

In [29]:
n_obs = (df["Control Cookies"] + df["Experiment Cookies"]).sum()
n_control = df["Control Cookies"].sum()

In [10]:
p = 0.5
sd = np.sqrt(p*(1-p) * n_obs)
z_score = ((n_control + 0.5) - p * n_obs) / sd
p_values = 2 * stats.norm.cdf(z_score)

In [11]:
p_values

np.float64(0.1074929405013041)

Looking at the test result, the H0 can't be rejected even on a $\alpha$=10% level. <br>
Hence, there is a high probability the number of visitors of the homepage in the treatment group is equally high as the number of visitors of the homepage in the control group.

## A/B Testing: the evaluation metric

The goal is to verify if the structure of the homepage has a significant effect on the:
* Download Rate and
* the purchasing rate

### Download Rate

Our goal is to check if the download rate is significantly higher with the newly structured home page than with the old structure 

Mathematically spoken:

$H_0: \, p_{treat} - p_{control} = 0$ <br>
$H_1: \, p_{treat} - p_{control} > 0$

In [12]:
df_test = df.copy() 

> Calculate the relevant metrics to do A/B Testing

In [15]:
n_control_cnt = df_test["Control Cookies"].sum()
n_exper_cnt = df_test["Experiment Cookies"].sum()
n_obs_cnt = n_control_cnt + n_exper_cnt

print(f"Control obs: {n_control_cnt}")
print(f"Exper. obs: {n_exper_cnt}")
print(f"Total obs: {n_obs_cnt}")

Control obs: 46851
Exper. obs: 47346
Total obs: 94197


In [16]:
n_control_downloads = df['Control Downloads'].sum()
n_experiment_downloads = df['Experiment Downloads'].sum()
n_total_downloads = n_control_downloads + n_experiment_downloads

print(f"Control downloads: {n_control_downloads}")
print(f"Exper. downloads: {n_experiment_downloads}")
print(f"Total downloads: {n_total_downloads}")

Control downloads: 7554
Exper. downloads: 8548
Total downloads: 16102


In [17]:
p_control_downloads = n_control_downloads / n_control_cnt
p_exper_downloads = n_experiment_downloads / n_exper_cnt
p_total_downloads = (n_control_downloads + n_experiment_downloads) / (n_control_cnt + n_exper_cnt)

print(f"Control downloads rate: {p_control_downloads}")
print(f"Exper. downloads rate: {p_exper_downloads}")
print(f"Total downloads rate: {p_total_downloads}")

Control downloads rate: 0.16123455209067042
Exper. downloads rate: 0.180543234908968
Total downloads rate: 0.1709396265273841


> Calculate the test-statistic and the p-value

In [18]:
#calc SE
se_p = np.sqrt(p_total_downloads * (1-p_total_downloads) * (1 / n_control_cnt + 1 / n_exper_cnt))

#calc z score
z_score = (p_exper_downloads - p_control_downloads) / se_p

print(f"Standard Error: {se_p}")
print(f"Z-Score: {z_score}")
print('p value: ', 1 - stats.norm.cdf(z_score))

Standard Error: 0.0024531940948456393
Z-Score: 7.870833726066236
p value:  1.7763568394002505e-15


> Check result via statsmodels

In [32]:
d0 = 0  # assume there is no difference between two groups
count = [n_experiment_downloads, n_control_downloads]
nobs = [n_exper_cnt, n_control_cnt]

zstat, pval = proptests.proportions_ztest(
    count,
    nobs,
    value=d0,  # null hypothesis
    alternative="larger",
    prop_var=False,  # pooled
)
zstat, pval

(np.float64(7.870833726066236), np.float64(1.7614279636728079e-15))

Looking at the tests, we can reject the H0 and so assume that the download rate is significantly higher with the new structure as with the old one.

### Licence

Our goal is to check if the licence purchase rate is significantly higher with the newly structured home page than with the old structure 

Mathematically spoken:

$H_0: \, p_{treat} - p_{control} = 0$ <br>
$H_1: \, p_{treat} - p_{control} > 0$

In [19]:
df_licences = df[df["Day"]<22]

n_control = df_licences["Control Cookies"].sum()
n_exper = df_licences["Experiment Cookies"].sum()

n_total = n_control + n_exper

print(f"n_control: {n_control}")
print(f"n_experiment: {n_exper}")
print(f"n_total: {n_total}")

n_control: 33758
n_experiment: 34338
n_total: 68096


> Calculate the relevant metrics to do A/B Testing

In [20]:
n_control_licences = df_licences['Control Licenses'].sum()
n_experiment_licences = df_licences['Experiment Licenses'].sum()
n_total_licences = n_control_licences + n_experiment_licences

print(f"Control Licences: {n_control_licences}")
print(f"Exper. Licences: {n_experiment_licences}")
print(f"Total Licences: {n_total_licences}")

Control Licences: 443
Exper. Licences: 456
Total Licences: 899


In [35]:
p_control_licences = n_control_licences / n_control
p_exper_licences = n_experiment_licences / n_exper
p_total_licences = (n_control_licences + n_experiment_licences) / (n_control + n_exper)

print(f"Control licence purchase rate: {p_control_licences}")
print(f"Exper. licence purchase rate: {p_exper_licences}")
print(f"Total licence purchase rate: {p_total_licences}")

Control licence purchase rate: 0.00945550788670466
Exper. licence purchase rate: 0.013279748383714835
Total licence purchase rate: 0.011072928598701794


> Calculate the test-statistic and the p-value

In [33]:
#calc SE
se_p = np.sqrt(p_total_licences * (1-p_total_licences) * (1 / n_control_cnt + 1 / n_exper_cnt))

#calc z score
z_score = (p_exper_licences - p_control_licences) / se_p

print(f"Standard Error: {se_p}")
print(f"Z-Score: {z_score}")
print('p value: ', 1 - stats.norm.cdf(z_score))

Standard Error: 0.0007437905264923841
Z-Score: 0.21099092481457538
p value:  0.41644717645292517


> Check result via statsmodels

In [34]:
se_p = np.sqrt(p_total_licences * (1-p_total_licences) * (1/n_control + 1/n_exper))
z_score = (p_exper_licences - p_control_licences) / se_p

p_val = 1- stats.norm.cdf(z_score)

print("z-score: {z}".format(z=z_score))
print("p-value from z-score: {p}".format(p=p_val))

z-score: 0.1935439350275868
p-value from z-score: 0.4232665006434394


Looking at the tests, we cannot reject the H0 and so we have to assume that the licence purchase rate is not significantly different between the new structure and the old one.